**Задание**

Что нужно делать?

Датасет ml-latest.

Вспомнить подходы, которые мы разбирали.

Выбрать понравившийся подход к гибридным системам.
Написать свою.

Материалы вы найдёте [здесь](https://github.com/ALKONDR/netology-recsys/blob/master/lecture-5/lecture-5-part-2.ipynb)

Загрузка библиотек и исходного датасета.

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811598 sha256=4b77de901c524b413c41a4999777512695a0746b0823b4d0191c29d61a9c109d
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import SVD, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import numpy as np
import pandas as pd

In [ ]:
!wget 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip' -O MovieLens.zip

--2023-08-20 14:07:22--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 955.28K  3.04MB/s    in 0.3s    

2023-08-20 14:07:23 (3.04 MB/s) - ‘MovieLens.zip’ saved [978202/978202]



In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


Для выполнения задания понадобятся датасеты movies и ratings.

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

Количество уникальных значений в датасетах ratings И movies:

In [ ]:
movies.nunique()

movieId    9742
title      9737
genres      951
dtype: int64

In [ ]:
ratings.nunique()

userId         610
movieId       9724
rating          10
timestamp    85043
dtype: int64

В датасете movies ID фильмов на 5 больше, чем названий фильмов. Удалим дубликаты.

In [ ]:
# Фильмы, которые имеют дубликаты ID:
duplicateRows = movies[movies.duplicated('title')]
duplicateRows

,movieId,title,genres
5601,26958,Emma (1996),Romance
6932,64997,War of the Worlds (2005),Action|Sci-Fi
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
9135,147002,Eros (2004),Drama|Romance
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller


In [ ]:
# Удаление дубликатов и проверка:
movies_new = movies.drop_duplicates(['title'])
movies_new.nunique()

movieId    9737
title      9737
genres      951
dtype: int64

In [ ]:
movies_with_ratings = movies_new.merge(ratings, on='movieId', how='outer')
movies_with_ratings.drop(columns=['timestamp', 'genres'], inplace=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100830 entries, 0 to 100847
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100830 non-null  int64  
 1   title    100830 non-null  object 
 2   userId   100830 non-null  float64
 3   rating   100830 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 3.8+ MB


In [ ]:
movies_with_ratings[:3]

,movieId,title,userId,rating
0,1,Toy Story (1995),1.0,4.0
1,1,Toy Story (1995),5.0,4.0
2,1,Toy Story (1995),7.0,4.5


Для построения гибридной системы воспользуемся алгоритмами библиотеки Surprise.

В ДЗ №2 при сравнении алгоритмов KNNWithMeans, KNNBasic, SVD и KNNBaseline лучшие результаты показали SVD и KNNBaseline, будем использовать их и технику "смешивание".

In [ ]:
# Подготовка датасета:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0)) # Минимальные и максимальные значения оценок
data = Dataset.load_from_df(dataset, reader)

In [ ]:
# Обучающая и тестовая выборки:
trainset, testset = train_test_split(data, test_size=.15, random_state=9)

In [ ]:
# Обучение алгоритма SVD:
algo_svd = SVD(n_factors=150, n_epochs=15, biased=True,
               init_mean=0, init_std_dev=0.01, lr_all=0.01,
               reg_all=0.02, random_state=9)
algo_svd.fit(trainset)

In [ ]:
# Обучение алгоритма KNNBaseline:
algo_knn = KNNBaseline(k=25, min_k=2)
algo_knn.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
accuracy.rmse(algo_svd.test(testset))

RMSE: 0.8525


0.8525328067454262

In [ ]:
accuracy.rmse(algo_knn.test(testset))

RMSE: 0.8571


0.8571039728929285

In [ ]:
def recommendation (user_id):

    user_movies = movies_with_ratings[movies_with_ratings.userId == user_id].title.unique() # Названия уже оцененных пользователем фильмов
    movies_to_score = movies_with_ratings.title.unique() # Названия всех фильмов

    scores_svd = []
    titles_svd = []
    scores_knn = []
    titles_knn = []

    for movie in movies_to_score:
        if movie in user_movies:
          continue
        # Предсказания оценок фильмов алгоритмом SVD:
        scores_svd.append(algo_svd.predict(uid = user_id, iid=movie).est)
        titles_svd.append(movie)

        # Предсказания оценок фильмов алгоритмом KNNBaseline:
        scores_knn.append(algo_knn.predict(uid = user_id, iid=movie).est)
        titles_knn.append(movie)

    # Вывод лучших 5-ти фильмов по "мнению" модели SVD:
    best_indexes_svd = np.argsort(scores_svd)[-5:]
    for i in reversed(best_indexes_svd):
            print(titles_svd[i], scores_svd[i].round(2))
    # Вывод второй лучшей пятерки фильмов по "мнению" модели KNNBaseline:
    best_indexes_knn = np.argsort(scores_knn)[-10:-5]
    for i in reversed(best_indexes_knn):
            print(titles_knn[i], scores_knn[i].round(2))

In [ ]:
# Проверка работы функции для отдельного пользователя:
recommendation (438)

Lawrence of Arabia (1962) 4.34
The Artist (2011) 4.32
Philadelphia Story, The (1940) 4.32
Streetcar Named Desire, A (1951) 4.29
Three Colors: Red (Trois couleurs: Rouge) (1994) 4.28
Victory (a.k.a. Escape to Victory) (1981) 4.98
Belle époque (1992) 4.94
Jonah Who Will Be 25 in the Year 2000 (Jonas qui aura 25 ans en l'an 2000) (1976) 4.91
Lamerica (1994) 4.89
Heidi Fleiss: Hollywood Madam (1995) 4.86
